# NLP Assignment 1 : 
-------------------------------------------------------------------
##                                        Ham or Spam - SMS classification
-------------------------------------------------------------------


### Aryaman Gautam
### J001
---------------------------------------------------------------------
----------------------------------------------------------------------


In [1]:
import pandas as pd

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Usha
[nltk_data]     Gautam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Usha
[nltk_data]     Gautam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
data = pd.read_csv('spam.csv')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
data.shape , data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


((5572, 5), None)

In [7]:
data.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [8]:
data = data.iloc[:,0:2]
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
data.describe()

,v1,v2
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [10]:
data['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [11]:
data['v1'] = data['v1'].replace({'ham': 0, 'spam': 1})

In [12]:
data.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
X = data['v2']
X[4]

"Nah I don't think he goes to usf, he lives around here though"

In [14]:
df = pd.DataFrame(X)
stop_words = set(stopwords.words('english'))


df['clean'] = df['v2'].apply(lambda x: ' '.join([word for word in x.split() if not word.lower() in (stop_words)]))
print(df)

                                                     v2  \
0     Go until jurong point, crazy.. Available only ...   
1                         Ok lar... Joking wif u oni...   
2     Free entry in 2 a wkly comp to win FA Cup fina...   
3     U dun say so early hor... U c already then say...   
4     Nah I don't think he goes to usf, he lives aro...   
...                                                 ...   
5567  This is the 2nd time we have tried 2 contact u...   
5568              Will �_ b going to esplanade fr home?   
5569  Pity, * was in mood for that. So...any other s...   
5570  The guy did some bitching but I acted like i'd...   
5571                         Rofl. Its true to its name   

                                                  clean  
0     Go jurong point, crazy.. Available bugis n gre...  
1                         Ok lar... Joking wif u oni...  
2     Free entry 2 wkly comp win FA Cup final tkts 2...  
3             U dun say early hor... U c already say...  
4

In [15]:
words = []
df = df['clean']

for i in range(0, df.shape[0]):
    result = text_to_word_sequence(df[i])
    words.extend(result)
    
len(set(words))

    
pd.DataFrame(words).value_counts().head(30)

u        1172
call      591
2         515
i'm       394
get       391
ur        385
gt        318
lt        316
4         316
ok        287
free      284
go        280
know      261
now       255
good      245
like      245
got       239
you       233
come      229
time      220
day       218
love      208
send      199
me        197
want      194
text      193
txt       177
it        176
one       174
going     173
dtype: int64

In [16]:
more_stopwords = set(['u','4','2',"i'm","i'll",'r','ur','n'])
updated_step_words = stop_words | more_stopwords

df1 = pd.DataFrame(df)

df1['clean_1'] = df1['clean'].apply(lambda x: ' '.join([word for word in x.split() if not word.lower() in (updated_step_words)]))
print(df1)

                                                  clean  \
0     Go jurong point, crazy.. Available bugis n gre...   
1                         Ok lar... Joking wif u oni...   
2     Free entry 2 wkly comp win FA Cup final tkts 2...   
3             U dun say early hor... U c already say...   
4               Nah think goes usf, lives around though   
...                                                 ...   
5567  2nd time tried 2 contact u. U �750 Pound prize...   
5568                      �_ b going esplanade fr home?   
5569           Pity, * mood that. So...any suggestions?   
5570  guy bitching acted like i'd interested buying ...   
5571                                    Rofl. true name   

                                                clean_1  
0     Go jurong point, crazy.. Available bugis great...  
1                           Ok lar... Joking wif oni...  
2     Free entry wkly comp win FA Cup final tkts 21s...  
3                 dun say early hor... c already say...  
4

In [17]:
ps = PorterStemmer()

def porter(text):
    text = nltk.word_tokenize(text) 
    text_set = []                   
    for i in text:                 
        text_set.append(ps.stem(i))  

    return " ".join(text_set)        

df1['text_steemed'] =df1['clean_1'].apply(porter)
df1

,clean,clean_1,text_steemed
0,"Go jurong point, crazy.. Available bugis n gre...","Go jurong point, crazy.. Available bugis great...","go jurong point , crazi .. avail bugi great wo..."
1,Ok lar... Joking wif u oni...,Ok lar... Joking wif oni...,ok lar ... joke wif oni ...
2,Free entry 2 wkly comp win FA Cup final tkts 2...,Free entry wkly comp win FA Cup final tkts 21s...,free entri wkli comp win fa cup final tkt 21st...
3,U dun say early hor... U c already say...,dun say early hor... c already say...,dun say earli hor ... c alreadi say ...
4,"Nah think goes usf, lives around though","Nah think goes usf, lives around though","nah think goe usf , live around though"
...,...,...,...
5567,2nd time tried 2 contact u. U �750 Pound prize...,2nd time tried contact u. �750 Pound prize. cl...,2nd time tri contact u . �750 pound prize . cl...
5568,�_ b going esplanade fr home?,�_ b going esplanade fr home?,�_ b go esplanad fr home ?
5569,"Pity, * mood that. So...any suggestions?","Pity, * mood that. So...any suggestions?","piti , * mood that . so ... ani suggest ?"
5570,guy bitching acted like i'd interested buying ...,guy bitching acted like i'd interested buying ...,guy bitch act like i 'd interest buy someth el...


In [18]:
Y = data['v1']
Y[0:5]

0    0
1    0
2    1
3    0
4    0
Name: v1, dtype: int64

In [19]:
cv = CountVectorizer()
X =cv.fit_transform(df1['text_steemed']).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [20]:
X_train, X_test,Y_train, Y_test = train_test_split(X,Y,stratify=Y, test_size= 0.2, random_state=1)

In [21]:
mnb = MultinomialNB()


mnb.fit(X_train,Y_train)
y_pred2 = mnb.predict(X_test)
print(accuracy_score(Y_test,y_pred2))
print(confusion_matrix(Y_test,y_pred2))

0.9874439461883409
[[958   8]
 [  6 143]]


In [22]:
model = SVC(kernel = 'rbf', random_state = 10)
model.fit(X_train,Y_train)
print(f'model fitted with accuracy ',(model.score(X_test,Y_test))*100)

model fitted with accuracy  97.75784753363229


In [23]:
tfidf = TfidfVectorizer()
X =tfidf.fit_transform(df1['text_steemed']).toarray()
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
X_train, X_test,Y_train, Y_test = train_test_split(X,Y,stratify=Y, test_size= 0.2, random_state=1)

In [25]:
lrc = LogisticRegression(solver='liblinear', penalty='l1')

def train_classifier(lrc,X_train,y_train,X_test,y_test):
    lrc.fit(X_train,y_train)
    y_pred = lrc.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    
    return accuracy,precision



train_classifier(lrc,X_train,Y_train,X_test,Y_test)

(0.9605381165919282, 0.9338842975206612)